In [1]:
#https://docs.llamaindex.ai/en/latest/examples/low_level/ingestion.html#

In [2]:
# !pip install weaviate-client

In [3]:
# !pip install llama_index
# !pip install openai

In [4]:
# !pip install --upgrade pydantic==1.10.0 typing-extensions==4.5.0

In [5]:
from llama_index.vector_stores import WeaviateVectorStore
from llama_index.schema import TextNode
from llama_index import VectorStoreIndex, StorageContext
import weaviate, os
from weaviate import EmbeddedOptions
from openai import OpenAI
import pandas as pd
import numpy as np
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from llama_index.embeddings import OpenAIEmbedding
from configparser import ConfigParser
import openai
import os

pd.set_option('display.max_colwidth',-1)

/tmp/ipykernel_138/1688819060.py:16: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth',-1)


## Set openAI

In [6]:
config=ConfigParser()
config.read('conf/conf.ini')
os.environ["OPENAI_API_KEY"] = config['openai']['apikey']

## Set weaviate

In [7]:
# !rm -rf ~/.local/share/weaviate

In [8]:
client = weaviate.Client(
    embedded_options=EmbeddedOptions(),
    # additional_headers={
    #     "X-OpenAI-Api-BaseURL": os.environ['OPENAI_API_BASE'],
    #     "X-OpenAI-Api-Key": openai.api_key,  # Replace this with your actual key
    # }
)
print(f"Client created? {client.is_ready()}")

Binary /root/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.23.0/weaviate-v1.23.0-Linux-amd64.tar.gz
Started /root/.cache/weaviate-embedded: process ID 205
Client created? True


{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-01-21T14:29:14Z"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-01-21T14:29:14Z"}
{"level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-01-21T14:29:14Z"}
{"level":"warning","msg":"Multiple vector spaces are present, GraphQL Explore and REST API list objects endpoint module include params has been disabled as a result.","time":"2024-01-21T14:29:14Z"}
{"action":"grpc_startup","level":"info","msg":"grpc server listening at [::]:50060","time":"2024-01-21T14:29:14Z"}
{"action":"restapi_management","level":"info","msg":"Serving weaviate at http://127.0.0.1:8079","time":"202

## Schema

In [10]:
# if client.schema.exists("MyCollection"):
#     client.schema.delete_class("MyCollection")

# schema = {
#     "class": "MyCollection",
#     "vectorizer": "none",
#     # "vectorIndexConfig": {
#     #     "distance": "cosine" # make sure to provide the distance metric to search through your vectors
#     # },
# }

# client.schema.create_class(schema)

# print("Successfully created the schema.")

## Insert data

In [11]:
df=pd.read_excel('data/riigikogu/ems_subset.xlsx')
embs=np.load('data/riigikogu/emsb.npy', allow_pickle=True)

/usr/lib/python3/dist-packages/openpyxl/compat/numbers.py:41: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  numpy.float,


In [12]:
df.shape, embs.shape

((1000, 1), (1000,))

In [13]:
data=[{'text':row['text_splitted'],'foo':i, 'vector':embs[i], } for i, row in df.iterrows()]

In [14]:
#https://docs.llamaindex.ai/en/stable/examples/low_level/oss_ingestion_retrieval.html#manually-construct-nodes-from-text-chunks
nodes = []
for idx, sample in enumerate(data):
    node = TextNode(
        text=sample['text'],
    )
    node.embedding=sample['vector']
    nodes.append(node)

## Create vector store

In [15]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
embed_model=OpenAIEmbedding()

service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=embed_model
)

In [16]:
# service_context = ServiceContext.from_defaults(llm=None,context_window=1000, embed_model='local')

In [17]:
# construct vector store
vector_store = WeaviateVectorStore(weaviate_client = client, index_name="Riigikogu", text_key="text")

# setting up the storage for the embeddings
storage_context = StorageContext.from_defaults(vector_store = vector_store)

# set up the index
index = VectorStoreIndex(nodes, storage_context = storage_context, service_context=service_context)

{"level":"info","msg":"Created shard riigikogu_UpmKCaaI6P3i in 1.551463ms","time":"2024-01-21T14:29:45Z"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-01-21T14:29:45Z","took":89486}


In [18]:
# client.batch.configure(batch_size=10)  # Configure batch

# # Batch import all objects
# # yes batch is an overkill for 5 objects, but it is recommended for large volumes of data
# with client.batch as batch:
#     for item in data:

#         properties = {
#          "text": item["text"],
#          "foo": item["foo"],
#         }

#         # the call that performs data insert
#         client.batch.add_data_object(
#          class_name="MyCollection",
#          data_object=properties,
#          vector=item["vector"] # your vector embeddings go here
#         )

# print("Data import complete")

In [19]:
# # Check number of objects
# response = (
#     client.query
#     .aggregate("MyCollection")
#     .with_meta_count()
#     .do()
# )

# print(response)

## Query engine

In [20]:
query_engine = index.as_query_engine()

In [24]:
query_str = "milliseid ettepanuekuid arutatakse, et riigieelarve tasakaalu saada?"

In [25]:
response = query_engine.query(query_str)

In [26]:
response.response

'Et riigieelarve tasakaalu saavutada, arutatakse erinevaid ettepanekuid. Üks võimalus on suurendada maksude kogumist, et tagada maksude tõhus kogumine. Samuti võidakse kaaluda kulude vähendamist või suurendada panustamist teenustesse, et tasakaalustada riigieelarvet.'

In [27]:
len(response.source_nodes)

2

In [28]:
response.source_nodes[0].text

'maksud ei ole ainuke võimalus asju mõjutada ja seejärel tulevad otsused rakendamised ja kindlasti järelhindamised mis võib-olla ei ole tänase diskussiooni teema aga ma proovin alustada algusest mis on üldse eesmärk kas eesmärk on saada rohkem raha riigieelarvesse kas probleemiks on väike maksukoormus mina arvan et meie eesmärk on saavutada võimalikult suur heaolu ja seda pidevalt suurendada ühiskonna heaolu saab tugineda üksnes majanduskasvule ja majanduskasv tuleneb ikka ainult ettevõtlusest nii et see on esimene asi mille peale tuleb mõelda järgmine'

In [29]:
response.source_nodes[1].text

'inimesed nii nagu igal pere eelarvel on ka riigieelarvel tulud ja kulud ning tasakaal võrrand on paigas kui tahame rohkem teenuseid siis tuleb ka rohkem panustada teisi võimalusi lihtsalt ei ole seejuures on riigieelarves suur fikseeritud kulude osakaal üle 80% kui taasiseseisvunud eestis alustati maksureformi siis oli ühtlase tulumaksu üks eesmärkidest tagada maksude tõhus kogumine sest iseseisvunud riigil puudus kogenud ametkond maksude kogumiseks selle puudumine oleks toonud kaasa maksude alalaekumise ning põrandaaluse majanduse osakaalu kasvu loodeti et madalamad maksud distsiplineerivad'

In [30]:
df[df.text_splitted.str.contains('need ei ole süsteemselt esitatud')]

,text_splitted


In [31]:
df[df.text_splitted.str.contains('programmilised ettekanded olemas peab rääkima ühest väikesest')]

,text_splitted
243,programmilised ettekanded olemas peab rääkima ühest väikesest maksust see on häbi ja siis tuleb üks kodanik kes kindlasti kaaperdab kogu selle debati ajakirjanduse jaoks jälle tema nägu jälle mis tema ütleb kuigi ta räägib läbisegi kõigest ja ei millestki ta räägib justkui ka maksusüsteemist aga täiesti süsteemitult igat tema väidet me oleme arutanud palju kordi ja selles mõttes on need teemad ju kõik maksunduses teada aga uitmõtetena neid lihtsalt ritta seada sobib küll kui sa esindad kedagi kui sa oled


In [32]:
df.sample(10)

,text_splitted
495,aitäh lugupeetud istungi juhataja mu küsimus on kaja kallasele peale valimisi esmaspäeval helistas mulle üks väga usin perearst kes teatas et temale on valitsuse poolt antud selline kohustus et nädalas tuleb vaktsineerida 120 inimest ja seda enne kui teie valitsuse korraldus jõustus küsimus on probleemne selles mõttes et see kohustus kehtib kõigile perearstidele tulemuseks on see et me ei hooli inimeste tervisest sellest kas nad on vastuvõtlikud sellele vaktsiinile või mitte sest on ju ka autoimmuunseid inimesi ja ka neid
485,merry aart palun sõna on teil
417,jaa kohalikud valimised pakkusid võimalust olla rohepöörde epitsentris pärnu maakonnas ja vaieldamatult sattusin ma ka diskussioonidesse kus räägiti just tuuleparkidest rahva küsimus on väga lihtne miks on nii et tootsi tuulepargi võimekus on välja arendatud on välja ehitatud alajaam liitumisvõimekus riigi rahaga on seda kompenseeritud aga tänaseni tuleb sealt null kilovatt-tundi elektrienergiat ehk tegelikult me maksame kinni oma tegematajätmist seda projekti on algatatud aastast 2009. täna oleme aastas 2021 ja olukord on selline et kaks riigiettevõtet käivad omavahel kohut paralleelselt
409,aitäh austatud peaminister euroopa rohelise kokkuleppe aluseks on lubadus et rohepööre on majanduslikult tulus ning sotsiaalselt õiglane ja et mitte kedagi ei jäeta maha tänases lehes ütleb arenguseire keskuse juhataja tea danilov et rohepöörde hetkel veel lõpuni teadmata majanduslik mõju on ilmne me ei tea tegelikult kuidas see meie majandust ja konkurentsivõimet mõjutab mõju on energiavarustuskindlusele samuti toob see kaasa üleminekuraskusi haavatavatele elanikkonnagruppidele tegelikult puudutab ta oma väljaütlemisega kahte põhilist alustala ehk ettevõtlussektorit majandusruumi pluss energiavarustuskindlust ehk meie julgeolekutemaatikat ja
262,kuuluvad oma sissetulekuga miinimumpalka saavad õpetajad kultuuritöötajad jt koalitsiooni esindaja rääkis siin energiamaksudest no me teame et gaasiaktsiis on tõusnud mitu aastat järjest ja tõuseb ka järgmisel aastal see ei ole mingisugune väike asi sest see puudutab konkurentsiteemat meie gaasiaktsiisi määr on kordades kõrgem kui teistel riikidel gaasiaktsiis mõjutab ka kodutarbijaid samuti kaugküttetarbijaid kuna piirkoormusi kaetakse ka gaasi abil nii et võimuliit peaks tegelikult vastama ka küsimusele millised on plaanid hinnatõusude leevendamiseks me ju oleme oma hinnatõusudega juba pikka aega
737,aitäh hea juhataja hea minister kurb oli kuulda et juriidilise teksti puhul peame me toetuma ministri usule kui marek strandberg küsis kas juriidiliselt on lasteaial mingisugust kaitset kaitsmaks ennast ülisuurte rühmade eest siiamaani oli liitrühmade puhul selge kaitse olemas siis te ütlesite et te usute seda järelikult on vastus et ei ole minul on küsimus mida küsis minu käest nüüdseks juba kolme lasteaia juhataja ta küsis väga konkreetselt palun küsige ministri käest et kui tuleb teadlik lapsevanem kes ütleb et
462,haeska tulistamise kohta kui närvid kaotanud politseinik vastutusvõimetu inimese lihtsalt kohapeal maha lasi ega me ei tea mis juhtunud on missugustele järeldustele on sisejuurdlus jõudnud küll teame me aga seda et politseiniku ja politseijaoskonna ülema vahel on sugulussidemed nüüd viimane näide 5. oktoobril politsei jõhkralt käitus asjasse mitte puutuva mitte tagaotsitava mitte menetletava mitte süüdistatava jaan hattoga purustas tema maja ukse pani ta käed raudu pani ta viieks tunniks raudus kätega põrandale ja purustas tema saunaukse seda kõike ettekäändel et
513,riigi tseremoniaalorkester aga mis puudutab kaitseväe orkestrit siis vormistatakse lepinguid nende lepingutega on niimoodi et nad võetakse tööle poole kohaga paljud pillimehed ei taha minna niimoodi poole kohaga tööle ja see tähendab seda et see orkester hakkab vaikselt lagunema ma olen rääkinud ka endiste orkestrijuhtidega et teha tseremoniaalorkestrit selleks läheb aastaid aega selleks et tema tattoo 'd rahvusvaheline tunnustatus kõik säiliks ja 

## Question which needs aggregation

In [33]:
query_str = "kui mitmes tekstis mainitakse maksupoliitikat?"
response = query_engine.query(query_str)
response.response

'Maksupoliitikat mainitakse ühes tekstis.'

In [34]:
df[df.text_splitted.str.contains('maksupoliitika')]

,text_splitted
79,kohustus seda tulu mis nad tänu sellele saavad inimestega jagada kas see on maksupoliitika küsimus või see on pigem autoriõiguse küsimus
265,austatud eesistuja head kolleegid kuulates täna neid seisukohti küsimusi ja kogu arutelu tekkis mul mõte selleks et liikuda edasi õige pisut tagasi vaadata võib-olla keskenduks sellele miks meil on väga raske teha koostööd näiteks reformierakonnaga esiteks meil on täiesti erinevad seisukohad maksupoliitikas me teame mida kujutab endast reformierakonna lihtne maksusüsteem see on olnud tulumaksuprotsendi järjekindel langetamine 26%-lt 20%-le see tähendabki et võidavad suurepalgalised kusjuures võidavad väga-väga palju näiteks kui inimene teenib palka 500 eurot kuus siis ta võidab aastas ühekordse
861,austatud juhataja seda eelnõu arutas rahanduskomisjon 5. aprillil s.a istungil tutvustas eelnõu algatajate esindajana rahandusministeeriumi maksupoliitika osakonna peaspetsialist iiris põldma nagu minister õigesti märkis on eelnõu eesmärk tagada eesti valmisolek eurole üleminekuks riigilõivuseadus sätestab eurodes kõik eestis võetavate riigilõivude määrad mis seni on väljendatud kroonides siin ei ole sisulisi muudatusi ega poliitilisi otsuseid ühe või teise riigilõivumäära korrigeerimiseks ääremärkusena mainin et selle tulemuseks on muidugi see et kui riigilõivud on siiani üldjuhul olnud üsna ümmargused summad siis paraku nüüd kuna


## Play with response syntheziser, node processors, retrievers

In [25]:
#https://nanonets.com/blog/llamaindex/

In [21]:
from llama_index import get_response_synthesizer
from llama_index.indices.postprocessor import SimilarityPostprocessor
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.postprocessor import SentenceTransformerRerank

# Set up the synthesizer
my_synthesizer = get_response_synthesizer(response_mode="tree_summarize")

In [31]:
#reranker to rank retrieved nodes
rerank = SentenceTransformerRerank(
    model="cross-encoder/ms-marco-MiniLM-L-12-v2", top_n=5
)

In [30]:
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=20,
)

In [32]:
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=my_synthesizer,
    node_postprocessors=[
        SimilarityPostprocessor(similarity_cutoff=0.7),
        #rerank
    ]
)

In [35]:
query_str = "milliseid ettepanuekuid arutatakse, et riigieelarve tasakaalu saada?"
response=query_engine.query(query_str)

In [36]:
response.response

'Arutatakse erinevaid ettepanekuid, et saavutada riigieelarve tasakaalu. Üheks ettepanekuks on keskkonnamaksu kasutamine ettevõtete tööjõumaksete vähendamiseks ning seejärel otsustada, kuidas rahastada sotsiaalseid teenuseid. Lisaks maksudele võib kaaluda ka ettevõtluse arendamist, näiteks maavarade väärindamise kaudu. Samuti mainitakse kulude kokkuhoiu võimalust.'

In [37]:
query_str = "milliseid teemade üle arutletakse?"
response=query_engine.query(query_str)
response.response

'Teemade üle arutletakse kliima ja keskkonna, maksude ja majanduskasvu ning kulude kokkuhoiu teemadel.'

In [32]:
response.response

'Teemade üle arutletakse kliimaneutraalsuse saavutamiseks vajalike kulutuste, ökoloogilise maksuraamistiku muutmise, luksusautode maksustamise, maksusüsteemi, tarbimismaksude, majanduskasvu, ettevõtluse, keskkonnaprobleemide, kliima ja keskkonna muutuste ning planeeritud lõikuste edasilükkamise teemadel.'

In [35]:
query_str = "milliseid seisukohti esitletakse ökoloogilise maksuraamistiku muutmiseks? Vasta eesti keeles"
response=query_engine.query(query_str)

In [36]:
response.response

'Ökoloogilise maksuraamistiku muutmiseks esitletakse mitmeid seisukohti. Üks seisukoht on, et ökoloogiliselt puhastele toiduainetele tuleks kehtestada käibemaksuvabastus. Samuti arvatakse, et varade maksustamist tuleks suurendada ja keskmise ja väikese sissetulekuga inimeste palgalt võetavaid makse vähendada. Lisaks räägitakse ka rikkuse suuremast maksustamisest ning küsitakse, mida ja kuidas maksustada, näiteks kinnisvara või aktsiaid ja osakuid. Mõned inimesed toetavad ka keskkonnasõbralike valikute soodustamist läbi maksusoodustuste.'

In [39]:
response.get_formatted_sources()

'> Source (Doc id: e87c97f1-eaeb-46e7-a09d-e8060169b3ce): lahendus on käibemaksuvabastus ökoloogiliselt puhastele toiduainetele järgmine teema varade maksu...\n\n> Source (Doc id: f17c5410-0c54-478a-9050-d05e692828e3): kui siis paar sekundit jah see teema on ülioluline ja me peaksime seda tõsiselt arutama mitte nae...\n\n> Source (Doc id: cac25036-c6c3-465f-a434-2877cd08188c): vabandama kui inimene tuleb ja ütleb et keskkonnasõbralikkuse huvides peab rohkem maksustama aga ...\n\n> Source (Doc id: 0de8761b-55c0-43e2-962f-0d42ec1f320b): tsiteeritud lause mis tõi meid siia ei vii meid enam edasi. kehtib täielikult ka eesti maksusüste...\n\n> Source (Doc id: 740d6392-2d4e-4d49-8f4c-bb4fe527a59b): näiteks kümme aastat tagasi olid ökotoodete letid kauplustes nii pikad kas me rääkisime keskkonna...\n\n> Source (Doc id: ad42e26a-f819-4239-9a86-a439f426c9a5): täna digimaksustamisest ei räägi õrnalt puudutan mõnesid maksumäärasid aga ma ei taha neid konkre...\n\n> Source (Doc id: 40a0ef42-d4ed-4

In [42]:
response.source_nodes[0].text

'lahendus on käibemaksuvabastus ökoloogiliselt puhastele toiduainetele järgmine teema varade maksustamine minu arvates on oluline vähendada veelgi palgalt võetavaid makse keskmise ja väikese sissetulekuga inimestel ning suurendada varadelt võetavaid makse ja maksustada rohkem suuremaid tulusid palju räägitakse viimasel ajal mitte ainult tulude vaid ka rikkuse suuremast maksustamisest see oleks ühiskonna kui terviku seisukohast väga õige ja õiglane küsimus on ainult mida ja kuidas maksustada kas kinnisvara või ka aktsiaid ja osakuid või rikkust tervikuna nagu soovitab ameerika ühendriikide demokraatliku partei presidendikandidaat'

In [38]:
query_str = "kui mitmes tekstis mainitakse maksupoliitikat?"
response=query_engine.query(query_str)
response.response

'Maksupoliitikat mainitakse kolmes tekstis.'

In [42]:
query_str = "kui mitmes tekstis mainitakse majandust?"
response=query_engine.query(query_str)
response.response

'Majandust mainitakse mitmes tekstis.'

In [43]:
query_str = "kui mitmes tekstis mainitakse majandust? Vasta arvuga"
response=query_engine.query(query_str)
response.response

'Majandust mainitakse 7 tekstis.'

In [45]:
response.get_formatted_sources()

'> Source (Doc id: a7c04550-e653-4733-9b0b-ee8d1e661ba1): ja kogu lugu puutumus eesti majandusega on null täna on nagu täitsa mõttetu siin firma käivitada ...\n\n> Source (Doc id: 3c6c8647-4570-46a3-9011-cf5bbbd8f55f): peaminister aga võttis taas julgelt sõna nimetades põlglikult inimarengu aruannet pelgalt heaks l...\n\n> Source (Doc id: 5847e994-f21b-48e0-aca3-092d249fd460): soovivad või mida ettevõtlus vajab alustame sellest et ilma ettevõtluseta ei oleks ju vaja rääkid...\n\n> Source (Doc id: 1b5de690-b3f2-4866-8ba6-2f49792fbfd8): olgu need julgeolek haridus tervishoid oma vaeste eest hoolitsemine või teedevõrgu edendamine kui...\n\n> Source (Doc id: 8c7bd1be-43d4-4f9c-9df4-e2e70e7e61e2): tõsta et eelarvetulusid mingil moel kasvatada raha on vahepeal ära raisatud ja samas te ütlete et...\n\n> Source (Doc id: 0fb4140a-8d4a-45c6-a373-d23d11122e16): aitäh küsimuse eest ma ei mõelnud ainult kinnisvara vaid sellist üldist rikkust see pole üksnes s...\n\n> Source (Doc id: 05979afd-bbde-4

In [46]:
len(response.source_nodes)

20

## Return only nodes

In [50]:
response_synthesizer = get_response_synthesizer(response_mode='no_text')

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=40,
)


query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[
        SimilarityPostprocessor(similarity_cutoff=0.7),
        rerank
    ]
)

In [53]:
response = query_engine.query("kui mitmes tekstis mainitakse majandust?")
response.response

''

In [54]:
len(response.source_nodes)

5

In [55]:
[node.text for node in response.source_nodes]

['austatud koosoleku juhataja head kolleegid kui siin maailmas üldse on midagi kindlat siis on need kaks asja nagu te teate maksud ja surm seda on öelnud usa riigimees benjamin franklin ekre kui parempoolse ja konservatiivse erakonna jaoks ei ole maksud mitte ainult vahend riigieelarve täitmiseks vaid ka majanduse juhtimiseks majanduskasv ei ole meie jaoks mitte abstraktne number vaid mõõdupuu mis näitab inimeste ja ettevõtete kasvavat jõukust majanduskasv on vajalik sest ühiskond vajab lisaraha nende ühishüvede tagamiseks milleks riik on loodud',
 'täna seda pigem naeruvääristati tegelikult tuleks inimestest rääkida no kelle jaoks seda majanduse konkurentsivõimet ikka on vaja kui mitte inimeste jaoks just nimelt inimeste jaoks seda vaja on teiseks muidugi töö maksustamine on meil kõrge see on tõsiasi aga nende suurte positsioonide võtmisel tuleb ju katteallikad leida täna me tegelikult ei saanud kinnitust et oleks võimalik varamaksudega asendada olulist osa praegustest maksudest ei sa

In [52]:
#no rerank
query_engine_no_rerank = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[
        SimilarityPostprocessor(similarity_cutoff=0.7),
        #rerank
    ]
)

response = query_engine_no_rerank.query("kui mitmes tekstis mainitakse majandust?")
response.response
[node.text for node in response.source_nodes]

['ja kogu lugu puutumus eesti majandusega on null täna on nagu täitsa mõttetu siin firma käivitada ostan parem börsilt aktsiaid on palju lihtsam elu kui niimoodi vaadata mis puutub üleeuroopalistesse maksudesse siis ma arvan et ka sellesama rikkuse maksustamisel võiks mõelda üleeuroopalise maksu peale kui ameerika näiteks hakkab selles suunas liikuma üleeuroopalise maksu korral ei teki üheski riigis mingit eelistust nii et rikkad inimesed läheksid näiteks prantsusmaalt kusagile naaberriiki või mujale see võiks olla lisaks digimaksudele',
 'soovivad või mida ettevõtlus vajab alustame sellest et ilma ettevõtluseta ei oleks ju vaja rääkida maksudebatist kui ei ole ettevõtlust ei ole tulu mida ümber jagada tänaselt arutelult mina ootasin et rohkem räägitakse sellest kuhu on vaja raha suunata milleks oleks vaja raha juurde saada ja teine pool kas riigieelarves on kuluartikleid mis võib-olla sellises hulgas raha ei vaja oluline on ikkagi vaadata kust king kõige rohkem pigistab aastast aastas

In [56]:
response = query_engine.query("millised tekstid räägivad riigikaitse tugevdamisest")
[node.text for node in response.source_nodes]

['toimepidevusele vastab tõesti tõele et me oleme eraldanud kaitse-eelarvele rohkem raha kui kunagi varem järgmisel aastal kasvab kaitse-eelarve 104 miljonit eurot ja see on kindlasti hea meie kaitsevõime tugevdamiseks seda milliseid samme täpselt astuda kaitsevõimekuse tõstmisel saame arutada ka riigikaitsekomisjonis et mis on need konkreetsed sammud aga ma arvan et oluline on nii see et oleks võimekus tehnika näol kui ka see et kaitsevägi toimib ja ei jää haigestumiste tõttu nendest ülesannetest kõrvale',
 'programmilised ettekanded olemas peab rääkima ühest väikesest maksust see on häbi ja siis tuleb üks kodanik kes kindlasti kaaperdab kogu selle debati ajakirjanduse jaoks jälle tema nägu jälle mis tema ütleb kuigi ta räägib läbisegi kõigest ja ei millestki ta räägib justkui ka maksusüsteemist aga täiesti süsteemitult igat tema väidet me oleme arutanud palju kordi ja selles mõttes on need teemad ju kõik maksunduses teada aga uitmõtetena neid lihtsalt ritta seada sobib küll kui sa es

In [57]:
response = query_engine_no_rerank.query("millised tekstid räägivad riigikaitse tugevdamisest")
[node.text for node in response.source_nodes]

['tänase päevakorra järgmine punkt on vabariigi valitsuse algatatud riigilõivuseaduse eelnõu esimene lugemine palun kõnetooli taas rahandusminister jürgen ligi',
 'peaminister aga võttis taas julgelt sõna nimetades põlglikult inimarengu aruannet pelgalt heaks lugemismaterjaliks teeme sellest järelduse et valitsusliitu ei huvita tegelikkus mõttevahetus ega analüüsid ainus soov on oma kavatsused iga hinna eest läbi suruda mõnikord lõõbitakse et on olemas kolme liiki valet väike vale suur vale ja statistika ometi tasub teinekord arvandmetesse süveneda tänase eesti kohta räägib statistika et 1/7 meestest ja 1/3 naistest hakkab enne tegelikku pensioniikka jõudmist saama kas töövõimetus- või sooduspensioni kui eeldada et pensioniiga tõstetakse toob',
 'läheme seitsmenda päevakorrapunkti juurde milleks on vabariigi valitsuse algatatud noorsootöö seaduse eelnõu esimene lugemine palun ettekandeks riigikogu kõnetooli haridus- ja teadusminister tõnis lukase',
 'järgmine punkt tänases päevakorras 

{"level":"info","msg":"Created shard riigikogu_meta_index_12OhX4kMA6CS in 3.596257ms","time":"2024-01-21T15:38:10Z"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-01-21T15:38:10Z","took":66910}
{"action":"lsm_memtable_flush","class":"Riigikogu_meta_index","error":"switch active memtable: init commit logger: open /root/.local/share/weaviate/riigikogu_meta_index/12OhX4kMA6CS/lsm/objects/segment-1705851998907240202.wal: no such file or directory","index":"riigikogu_meta_index","level":"error","msg":"flush and switch failed","path":"/root/.local/share/weaviate/riigikogu_meta_index/12OhX4kMA6CS/lsm/objects","shard":"12OhX4kMA6CS","time":"2024-01-21T15:46:38Z"}
{"action":"lsm_memtable_flush","class":"Riigikogu_meta_index","error":"switch active memtable: init commit logger: open /root/.local/share/weaviate/riigikogu_meta_index/12OhX4kMA6CS/lsm/objects/segment-1705851999007788362.wal: no su

## Arxiv agent

In [2]:
# !pip install llama-hub
# !pip install arxiv

#https://llamahub.ai/

In [10]:
from llama_hub.tools.arxiv import ArxivToolSpec
from llama_index.agent import OpenAIAgent

tool_spec = ArxivToolSpec()

agent = OpenAIAgent.from_tools(tool_spec.to_tool_list())

response=agent.chat('Whats going on with the superconductor lk-99')
response.response

/usr/local/lib/python3.9/dist-packages/llama_hub/tools/arxiv/base.py:35: ResourceWarning: unclosed <ssl.SSLSocket fd=72, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('10.36.202.193', 47864), raddr=('128.84.21.203', 443)>
  for result in search.results():


'I found some information about the superconductor LK-99. \n\nOne paper titled "Synthesis of possible room temperature superconductor LK-99:Pb$_9$Cu(PO$_4$)$_6$O" discusses the quest for room-temperature superconductors and mentions that a Korean group synthesized a Lead Apatite-based compound called LK-99, which showed a transition temperature (T$_c$) above 400$^\\\\circ$K. The paper mentions that the compound exhibited promising signatures of superconductivity in terms of resistivity and diamagnetism at T$_c$. However, the heat capacity did not show an obvious transition at T$_c$. The paper reports the synthesis of polycrystalline samples of LK-99 and confirms the phase through X-ray diffraction measurements. The paper concludes that the current results do not approve the appearance of bulk superconductivity at room temperature, but further studies with different heat treatments are underway. [Link to the paper](http://arxiv.org/pdf/2307.16402v2)\n\nAnother paper titled "Superfluid d

In [11]:
response=agent.chat('what are the latest developments in automatic speech recognition')
response.response

/usr/local/lib/python3.9/dist-packages/llama_hub/tools/arxiv/base.py:35: ResourceWarning: unclosed <ssl.SSLSocket fd=72, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('10.36.202.193', 36546), raddr=('128.84.21.203', 443)>
  for result in search.results():


'Here are some of the latest developments in automatic speech recognition:\n\n1. [Scaling Laws of Synthetic Images for Model Training](http://arxiv.org/pdf/2312.04567v1): This paper explores the use of synthetic images generated by text-to-image models for training supervised models such as image classifiers and CLIP. The study investigates the scaling behavior of synthetic images and identifies factors that affect their effectiveness in training. The findings suggest that scaling synthetic data can be particularly effective in scenarios with limited real image data or when the evaluation dataset differs significantly from the training data.\n\n2. [Gen2Det: Generate to Detect](http://arxiv.org/pdf/2312.04566v1): This work presents Gen2Det, a modular pipeline for creating synthetic training data for object detection. By leveraging state-of-the-art grounded image generation methods, Gen2Det simplifies the process of generating scene-centric images for object detection. The paper also pro

## Ideas how to use:
 - make query, ask some overview of what they talk about x and let embedding find samples and LLM gen answer
 - 